# Vehicle Routing Problem
El Problema de Enrutamiento de Vehículos (Vehicle Routing Problem, VRP) es un problema de optimización combinatoria que implica encontrar la ruta más optima para un conjunto de vehículos que deben visitar un conjunto de ciudades, atendiendo la demanda de cada ciudad, y regresar al depósito inicial.

![f1](img/f1.png)

## Objetivo
El objetivo es minimizar el costo total,que vendria ser la distancia total sumando cada distancia de las rutas de los "m" vehiculos.
## Formulación Matemática
El modelo matematico podemos expresarlo de la siguiente manera:

**Parámetros:**
- $ n: \text{Numero de ciudades o nodos en el grafo}.$
- $ C: \text{Conjunto de ciudades, }C={0,1,2,...,n-1}$. 
- $c_{ij}: \text{Costo para ir de la ciudad } (i) \text{ a la ciudad } (j)$.
- $m:  \text{Número de vehículos en el almacén.}$


**Variables de decisión:**
$$ x_{ij} = \begin{cases} 1 & \text{si se elige la arista de la ciudad } i \text{ a la ciudad } j \\ 0 & \text{en caso contrario} \end{cases} $$
$$ v_{ik} = \begin{cases} 1 & \text{si se visita el nodo } i \text{ por el vehiculo } k \\ 0 & \text{en caso contrario} \end{cases} $$

**Función Objetivo:**
$$ \text{Minimizar} \quad Z = \sum_{k = 1}^{m} \sum_{j = 1}^{n} \sum_{i = 1, j \neq i}^{n} c_{ij} \cdot x_{ij} \cdot v_{ik} $$

**Restricciones:**

***Sean:*** 
$$ V = {1, 2, 3, ..., n}$$
$$ S = {1, 2, 3, ...., m}$$

1. $ \text{Cada ciudad } i  \text{ debe ser visitada exactamente una vez por un vehiculo } k. $
$$\forall i \in  V$$
$$ \sum_{k = 1}^{m} v_{ik} = 1  $$

2. $\text{Cada vehiculo debe visitar al menos un nodo a parte del nodo de partida.}$
$$\forall k \in S$$
$$ \sum_{i = 1}^{n} v_{ik} \geq 2 $$

3. $\text{Cada nodo tiene una entrada y salida para cada vehiculo.}$

$$\forall k \in S, \forall j \in V \text{ tal que } v_{jk} = 1$$
$$ \sum_{i = 1, i \neq j}^{n} x_{ij} v_{ik} = 1$$
$$\forall k \in S, \forall i \in V \text{ tal que } v_{ik} = 1$$
$$ \sum_{j = 1, i \neq j}^{n} x_{ij} v_{jk} = 1$$


# Pseudocodigo

~~~python
    rutas optimas():
        Para cada permutacion `P` generada:
            Para cada distribucion `D` de la permutacion `P`:
                Si (es_posible_recorrer(`D`)):
                    costo_total = costo_rutas(`D`)
                    si (costo_total < costo_optimo):
                        costo_optimo = costo_total
                        ruta_optima =  `D`
        return costo_optimo, ruta_optima
~~~

# Implementación en Python 

In [2]:
import numpy as np

class VRP:
    # constructor
    def __init__(self, nCiudades, almacen, nVehiculos):
        self.nCiudades = nCiudades
        self.almacen = almacen
        self.nVehiculos = nVehiculos
        self.matriz_costo = np.full((nCiudades, nCiudades), np.inf)
    
    # metodos
    def agregar_arista(self, inicio, fin, distancia):
        self.matriz_costo[inicio][fin] = distancia
        self.matriz_costo[fin][inicio] = distancia

    def costo_ruta(self, ruta):
        costo_total = 0
        for i in range(len(ruta) - 1):
            costo_total += self.matriz_costo[ruta[i]][ruta[i + 1]]
        return costo_total
    
    def generar_permutaciones(self, lista):
        if len(lista) == 0:
            return [[]]
        permutaciones = []
        for i in range(len(lista)):
            resto = lista[:i] + lista[i+1:]
            for p in self.generar_permutaciones(resto):
                permutaciones.append([lista[i]] + p)

        return permutaciones
    
    def rutas_optimas(self):
        costo_optimo = np.inf
        ruta_optima = tuple()
        ciudades = list(range(self.nCiudades))
        ciudades.remove(self.almacen)

        for permutacion in self.generar_permutaciones(ciudades):

            for i in range(1, len(permutacion)):
                
                ruta_1 = [self.almacen] + permutacion[:i] + [self.almacen]
                ruta_2 = [self.almacen] + permutacion[i:] + [self.almacen]

                if len(ruta_1) > 2 and len(ruta_2) > 2: # Restriccion
                    costo_ruta_1 = self.costo_ruta(ruta_1)
                    costo_ruta_2 = self.costo_ruta(ruta_2)
                    costo_total = costo_ruta_1 + costo_ruta_2
                    
                    print("Permutacion actual: ", permutacion)
                    print(f"Ruta 1: {ruta_1} con Costo: {costo_ruta_1}")
                    print(f"Ruta 2: {ruta_2} con Costo: {costo_ruta_2}")
                    print(f"Costo total: {costo_total}")
                    
                    if costo_total < costo_optimo:
                        costo_optimo = costo_total
                        ruta_optima = [ruta_1[:-1], ruta_2[:-1]]

        return ruta_optima, costo_optimo


vrp = VRP(nCiudades=6, almacen=2, nVehiculos=2)
'''
aristas = [
    (0, 1, 1),
    (0, 2, 3),
    (0, 3, 2),
    (1, 2, 3),
    (1, 3, 5),
    (2, 3, 4)
] 
'''
# Ejemplo del ppt del profe - Tarea
aristas = [
    (0, 1, 4),
    (0, 2, 5),
    (0, 3, 2),
    (0, 4, 3),
    (1, 2, 3),
    (1, 3, 1),
    (1, 4, 2),
    (2, 3, 5),
    (3, 4, 6),
    (0, 5, 3),
    (1, 5, 4),
    (2, 5, 3),
    (2, 4, 4),
    (3, 5, 1),
    (4, 5, 2)
]
for inicio, fin, distancia in aristas:
    vrp.agregar_arista(inicio, fin, distancia)

ruta_optima, costo_optimo = vrp.rutas_optimas()
for a in ruta_optima:
    print(a[0], end=" --> ")
    for i in a[1:]:
        print(i, end=" --> ")
    print(a[0])
print(f'\nMejor costo: {costo_optimo}')

Permutacion actual:  [0, 1, 3, 4, 5]
Ruta 1: [2, 0, 2] con Costo: 10.0
Ruta 2: [2, 1, 3, 4, 5, 2] con Costo: 15.0
Costo total: 25.0
Permutacion actual:  [0, 1, 3, 4, 5]
Ruta 1: [2, 0, 1, 2] con Costo: 12.0
Ruta 2: [2, 3, 4, 5, 2] con Costo: 16.0
Costo total: 28.0
Permutacion actual:  [0, 1, 3, 4, 5]
Ruta 1: [2, 0, 1, 3, 2] con Costo: 15.0
Ruta 2: [2, 4, 5, 2] con Costo: 9.0
Costo total: 24.0
Permutacion actual:  [0, 1, 3, 4, 5]
Ruta 1: [2, 0, 1, 3, 4, 2] con Costo: 20.0
Ruta 2: [2, 5, 2] con Costo: 6.0
Costo total: 26.0
Permutacion actual:  [0, 1, 3, 5, 4]
Ruta 1: [2, 0, 2] con Costo: 10.0
Ruta 2: [2, 1, 3, 5, 4, 2] con Costo: 11.0
Costo total: 21.0
Permutacion actual:  [0, 1, 3, 5, 4]
Ruta 1: [2, 0, 1, 2] con Costo: 12.0
Ruta 2: [2, 3, 5, 4, 2] con Costo: 12.0
Costo total: 24.0
Permutacion actual:  [0, 1, 3, 5, 4]
Ruta 1: [2, 0, 1, 3, 2] con Costo: 15.0
Ruta 2: [2, 5, 4, 2] con Costo: 9.0
Costo total: 24.0
Permutacion actual:  [0, 1, 3, 5, 4]
Ruta 1: [2, 0, 1, 3, 5, 2] con Costo: 14.0

![f3](img/f2.png)